Instalações e importações necessárias

In [1]:
!pip install pydantic
!pip install pydantic[email]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.6/313.6 kB 8.6 MB/s eta 0:00:00


In [18]:
import enum
import hashlib
import re
from typing import Any, Self
from pydantic import (
    BaseModel,
    EmailStr,
    Field,
    field_serializer,
    field_validator,
    model_serializer,
    model_validator,
    SecretStr,
)

Criação de expressões regulares para validação A primeira garante que a senha tenha pelo menos oito caracteres, contendo letra maiúscula, minúscula e um número Já a segunda garante que o nome tenha apenas letras e no mínimo 2 caracteres



In [19]:
VALID_PASSWORD_REGEX = re.compile(r"^(?=.*[a-z])(?=.*[A-Z])(?=.*\d).{8,}$")
VALID_NAME_REGEX = re.compile(r"^[a-zA-Z]{2,}$")

Criação da classe Role, que agora inclui um User com valor 0, servindo como padrão

In [20]:
class Role(enum.IntFlag):
    User = 0
    Author = 1
    Editor = 2
    Admin = 4
    SuperAdmin = 8

Criação da classe User, agora o password possui o parâmetro exclude=True, que exclui o campo por padrão
e role agora tem o valor 0 e validate_default=True diz que o valor padrão deve passar pela validação

field_validator para "name", que valida que o nome seja compatível com a expressão regular citada acima e lança um erro caso seja inválido

field_validator para "role", que aceita o papel como inteiro, string ou instância, usando um dicionário de operações para converter o valor do tipo Role correto e lança um erro se o papel for inválido

model_validator mode before roda antes do processo de validação, verificando se o nome e a senha estão presentes nos dados. Além de garantir que a senha não contenha o nome do usuário, transformando a senha em um hashsha-256 como medida de segurança. Agora foi renomeado para user_pre para diferenciar do próximo validador

model_validator mode after é executado após todas as validações padrão e verifica uma regra, somente usuários com nome de "Arjan" irão poder ser admin e retorna a própria instância

field_serializer role é aplicado ao role quando o modo de é json e converte o objeto role ao seu nome ao invés de valor numérico

model_serializer mode wrap também é aplicado quando o modo de serialização é json e, se não houver inclusões ou exclusões específicas, retorna apenas name e role

In [30]:
class User(BaseModel):
    name: str = Field(examples=["Example"])
    email: EmailStr = Field(
        examples=["user@arjancodes.com"],
        description="The email address of the user",
        frozen=True,
    )
    password: SecretStr = Field(
        examples=["Password123"], description="The password of the user", exclude=True
    )
    role: Role = Field(
        description="The role of the user",
        examples=[1, 2, 4, 8],
        default=0,
        validate_default=True,
    )

    @field_validator("name")
    def validate_name(cls, v: str) -> str:
        if not VALID_NAME_REGEX.match(v):
            raise ValueError(
                "Name is invalid, must contain only letters and be at least 2 characters long"
            )
        return v

    @field_validator("role", mode="before")
    @classmethod
    def validate_role(cls, v: int | str | Role) -> Role:
        op = {int: lambda x: Role(x), str: lambda x: Role[x], Role: lambda x: x}
        try:
            return op[type(v)](v)
        except (KeyError, ValueError):
            raise ValueError(
                f"Role is invalid, please use one of the following: {', '.join([x.name for x in Role])}"
            )

    @model_validator(mode="before")
    @classmethod
    def validate_user_pre(cls, v: dict[str, Any]) -> dict[str, Any]:
        if "name" not in v or "password" not in v:
            raise ValueError("Name and password are required")
        if v["name"].casefold() in v["password"].casefold():
            raise ValueError("Password cannot contain name")
        if not VALID_PASSWORD_REGEX.match(v["password"]):
            raise ValueError(
                "Password is invalid, must contain 8 characters, 1 uppercase, 1 lowercase, 1 number"
            )
        v["password"] = hashlib.sha256(v["password"].encode()).hexdigest()
        return v

    @model_validator(mode="after")
    def validate_user_post(self, v: Any) -> Self:
        if self.role == Role.Admin and self.name != "Arjan":
            raise ValueError("Only Arjan can be an admin")
        return self

    @field_serializer("role", when_used="json")
    @classmethod
    def serialize_role(cls, v) -> str:
        return v.name

    @model_serializer(mode="wrap", when_used="json")
    def serialize_user(self, serializer, info) -> dict[str, Any]:
        if not info.include and not info.exclude:
            return {"name": self.name, "role": self.role.name}
        return serializer(self)

criação da função main, que define dados para um usuário válido, validando eles e criando o User, tendo diferentes modos de serialização como o modelo_dump para um dicionário python, mode json para um formato adequado JSON, o terceiro demonstra exclusão seletiva de campos e o último exclusão direta para dicionário, visto que possui um comportamento diferente

In [31]:
def main() -> None:
    data = {
        "name": "Arjan",
        "email": "example@arjancodes.com",
        "password": "Password123",
        "role": "Admin",
    }
    user = User.model_validate(data)
    if user:
        print(
            "The serializer that returns a dict:",
            user.model_dump(),
            sep="\n",
            end="\n\n",
        )
        print(
            "The serializer that returns a JSON string:",
            user.model_dump(mode="json"),
            sep="\n",
            end="\n\n",
        )
        print(
            "The serializer that returns a json string, excluding the role:",
            user.model_dump(exclude=["role"], mode="json"),
            sep="\n",
            end="\n\n",
        )
        print("The serializer that encodes all values to a dict:", dict(user), sep="\n")

In [32]:
if __name__ == "__main__":
    main()

The serializer that returns a dict:
{'name': 'Arjan', 'email': 'example@arjancodes.com', 'role': <Role.Admin: 4>}

The serializer that returns a JSON string:
{'name': 'Arjan', 'role': 'Admin'}

The serializer that returns a json string, excluding the role:
{'name': 'Arjan', 'email': 'example@arjancodes.com'}

The serializer that encodes all values to a dict:
{'name': 'Arjan', 'email': 'example@arjancodes.com', 'password': SecretStr('**********'), 'role': <Role.Admin: 4>}
